In [2]:
import bisect
import json

# Classe


class ArbreHuffman():
    def __init__(self, lettre, nbocc, g=None, d=None):
        """
        Constructeur
        """
        self.lettre = lettre
        self.nbocc = nbocc
        self.gauche = g
        self.droite = d

    def est_feuille(self) -> bool:
        # verification de la presence de fils droit ou gauche
        return not(self.gauche) and not(self.droite)

    def __lt__(self, other):
        return self.nbocc > other.nbocc  # renvoie une copie du nombre d'occurrence


# Fonctions
def lire_fichier(file):
    """
    Fonction qui prend en paramètre un fichier et le renvoie en tant que chaîne de caractère
    """
    with open(file, 'r') as txt:
        lignes = txt.read()
        assert lignes, f"Le fichier {file} est vide."
        return lignes


def ecrire_fichier(texte, file="texte.txt"):
    """
    Fonction qui prend en paramètre un fichier texte et renvoie le texte sous la forme d'une chaîne de caractère
    """
    with open(file, 'w') as txt:
        txt.write(texte)


def parcours(arbre, chemin_en_cours, dico):
    """
    Fonction qui prend en paramètre un arbre, une liste représentant le parcours et un dictionnaire. Cette fonction
    permet de modifier le dictionnaire pour qu'il ai l'ordre du parcours Huffman
    """
    assert arbre, "Arbre inexistant"
    if arbre.est_feuille():
        dico[arbre.lettre] = chemin_en_cours  # condition d'arrêt du parcours
    else:
        parcours(arbre.gauche, chemin_en_cours + [0], dico)  # appel récursif
        parcours(arbre.droite, chemin_en_cours + [1], dico)  # Appel récursif


def fusionne(gauche, droite):
    nbocc_total = gauche.nbocc + droite.nbocc  # calcul du nom d'occurrence total
    return ArbreHuffman(
        None,
        nbocc_total,
        gauche,
        droite)  # renvoie l'arbre final


def compte_occurences(texte: str) -> dict:
    """
    Fonction qui prend en paramètre un texte de type str et renvoie le nombre d’occurrences de chaque
    caractères
    """
    occ = dict()  # creation d'un dictionnaire pour les occurrences
    for car in texte:  # balayage pour chaque caractère dans le texte
        assert not(
            car) == "'", "Nous ne pouvons pas prendre en compte le caractère ' (guillemet)"
        if car not in occ:  # verification de la présence d'espace dans le texte
            occ[f"{car}"] = 0  # forçage de l'espace en caractère
        # incrementation du nombre d'occurrence des espaces
        occ[f"{car}"] = occ[f"{car}"] + 1
    return occ  # renvoie des occurrences


def construit_liste_arbres(texte: str) -> list:
    """
    Fonction qui prend en paramètre le texte en chaîne de caractère et renvoie une liste d'arbres ayant utiliser le
    parcours Huffman
    """
    dic_occurences = compte_occurences(
        texte)  # ajout du nombre d'occurrence dans un dictionnaire
    liste_arbres = []  # creation d'une liste
    for lettre, occ in dic_occurences.items(
    ):  # balayage pour chaque lettres dans les occurrences
        # ajout de l'ensemble des lettres et des occurrences dans la liste
        liste_arbres.append(ArbreHuffman(lettre, occ))
    return liste_arbres  # renvoie de la liste


def codage_huffman(texte: str) -> dict:
    """
    Fonction qui prend en paramètre le texte sous forme de chaîne de caractère et qui renvoie
    un dictionnaire ayant les carcactères en tant que clés et le nombre d'occurences en tant que valeur
    """
    liste_arbres = construit_liste_arbres(
        texte)  # créé une liste d'arbre à partir du texte
    liste_arbres.sort()  # triage des lettres de la liste par le nombre d'occurrence
    while len(liste_arbres) > 1:  # boucle s'arrêtant lorsque la liste est vide
        droite = liste_arbres.pop()
        gauche = liste_arbres.pop()  # ajout des valeurs de la liste aux fils de l'arbre
        # creation d'un nouvel arbre  en fusionnant les deux précédant
        new_arbre = fusionne(gauche, droite)
        # Ajout d'un élément à la liste, tout en conservant cette dernière
        # triée
        bisect.insort(liste_arbres, new_arbre)
    arbre_huffman = liste_arbres.pop()  # creation de l'arbre finale
    dico = {}  # ceration d'un dictionnaire
    parcours(arbre_huffman, [], dico)  # appel de la fonctin parcours
    encode_message(dico, texte)  # encode le dictionnaire
    return dico  # renvoie le dictionnaire encodé


def encode_message(dico, texte) -> str:
    """
    Encodage du message selon la méthode de compression de Huffman
    Cette fonction prend en paramètres le dictionnaire contenant les codes binaires de chaque caractère du texte,
    également passé en paramètres.
    """
    encoded = ""  # création de la variable
    for car in texte:  # balayage pour chaque caractère dans le texte
        # balayage pour chaque valeur dans dans le dictionnaire créé à partir
        # du texte
        for val in dico[car]:
            # incrémentation de la variable avec la valeur de chaque caractère
            encoded = encoded + f"{val}"
    sauvegarde(encoded, dico)  # sauvegarde du dictionnaire encodé
    return encoded  # renvoie de la variable contenant l'encodage


def sauvegarde(encoded, dico):
    """
    Fonction qui prend en paramètre les valeurs obtenues par la fonction
    encode_message dans des fichiers
    """
    with open("huffman.txt", "w") as file:
        file.write(encoded)  # création d 'un fichier contenant l'encodage
    with open("decodage.txt", "w") as dic:
        # création d'un fichier json contenant la version décodé
        dic.write(json.dumps(dico))


def decrypte(textfile, codefile):
    """
    Fonction qui prend en paramètre le code binaire du texte et les codes binaires associés
    aux caractères du texte et qui renvoie le texte de départ
    """
    decrypted, current = [], ""  # création d'une liste vide
    with open(textfile) as txt:
        texte = txt.read()  # lecture du texte
    with open(codefile) as dic:
        dictionnaire = dic.read()  # lecture du dictionnaire
    # remplacement du caractère " par ' afin de convertir le fichier en json
    dico = dictionnaire.replace("'", '"')
    dico = json.loads(dico, encoding='utf-8')  # chargement du fichier json
    # inversion des clés du dictionnaire avec les valeurs afin de faciliter le
    # décodage
    reverse_code = dict(zip(["".join([str(y) for y in v])
                        for v in dico.values()], dico.keys()))
    for c in texte:  # balayage pour chaque caractère dans le texte
        current += c  # ajout du caractère dans la variable "current"
        if current in reverse_code:  # vérification de la présence du caractère dans le dictionnaire inversé
            # ajout du caractère dans la liste
            decrypted.append(reverse_code[current])
            current = ""  # remise a zéro de "current" pour passer au caractère suivant
    return "".join(decrypted)  # renvoie du fichier décodé


def execution():
    """
    Fonction permettant l'exécution de l'ensemble des autres fonctions nécessaires au bon fonctionnement du programme.
    Cette fonction ne nécessite pas de paramètre, et fonctionne grâce aux appels de fonction 'input()'
    """
    dem0 = input(
        "Tapez 1 pour utiliser l'algorithme d'encodage\nTapez 2 pour utiliser celui du décryptage \n>>> ")
    if dem0 == "1":  # Si l'utilisateur choisit l'encodage
        dem1 = input(
            "Quel est le nom du fichier texte (+ extension .txt) ? >>> ")
        dem2 = input(
            "Souhaitez-vous écrire dans le fichier maintenant (O / N) ? >>> ")
        if dem2 == "O":  # Si l'utilisateur veut écrire dans le fichier
            dem3 = input("Quel est le texte à insérer ? >>> ")
            ecrire_fichier(dem3, dem1)
        texte = lire_fichier(dem1)
        print(
            "Voici le texte qui va être encodé et compressé selon la méthode d'Huffman\n\t",
            texte)
        dico = codage_huffman(texte)
        print(
            "\nVoici maintenant le dictionnaire utilisé pour la compression\n\t",
            dico)
        encode = encode_message(dico, texte)
        print("\nEt, pour finir, voici la version encodée et compressée\n\t", encode)
        dem4 = input(
            "Souhaitez-vous utiliser l'algorithme pour décrypter le message encodé à l'instant ? (O / N) >>> ")
        if dem4 == "O":
            dec = decrypte("huffman.txt", "decodage.txt")
            print("\nVoici ce qui a été décrypté : \n\t", dec)
    if dem0 == "2":  # Si l'utilisateur choisit le décodage
        print("De base, les fichiers contenant la version compressée et le dictionnaire de décryptage sont nommés respectivement 'huffman.txt' et 'decodage.txt'")
        dem5 = input("Avez-vous modifié ces noms ? (O / N) >>> ")
        if dem5 == "O":  # Si l'utilisateur a changé les noms des fichiers
            dem6 = input(
                "Quel est le nom du fichier où est stockée la version compressée (+ extension .txt) ? >>> ")
            dem7 = input(
                "Quel est le nom du fichier contenant le dictionnaire décryptage (+ extension .txt) ? >>> ")
            assert dem6 and dem7, "Fichier introuvable"
            dec = decrypte(dem5, dem6)
        else:  # Sinon, si les noms sont 'par défaut'
            dec = decrypte("huffman.txt", "decodage.txt")
        print("\nVoici ce qui a été décrypté : \n\t", dec)


# Programme principal
if __name__ == '__main__':
    # Assertions
    assert compte_occurences("ABRACADABRA") == {
        'A': 5, 'B': 2, 'R': 2, 'C': 1, 'D': 1}, "Erreur d'implémentation"
    test1 = ArbreHuffman("z", 3)
    assert test1.lettre == "z" and test1.nbocc == 3, "Erreur d'implémentation"

    # Exécution de l'encodage / décodage
    execution()

Tapez 1 pour utiliser l'algorithme d'encodage
Tapez 2 pour utiliser celui du décryptage 
>>> 2
De base, les fichiers contenant la version compressée et le dictionnaire de décryptage sont nommés respectivement 'huffman.txt' et 'decodage.txt'
Avez-vous modifié ces noms ? (O / N) >>> N

Voici ce qui a été décrypté : 
	 Salut la famille comment ça va ?


    On commence par importer les modules dont nous alons avoir besoin

    Ensuite, on définit la classe ArbreHuffman, qui nous permettrade définir nos noeuds comme des objets, et ainsi les manipuler plus facilement.
    Pour construire cet objet, nous avons besoin au minimum d'un caractère, et d'un nombre d'occurences de ce caractère. Il est également possible de définir un fils droite, et gauche.
    Dans cette classe, nous avons deux méthodes, la première s'intitule 'est_feuille', et renvoie un booléen pour indiquer si le noeud visé est une feuille. (True si oui, False si non). Cela repose sur la vérification de l'existence, ou non, de fils gauche et droite. Notre seconde méthode est la méthode spéciale __lt__. Elle nous permet de comparer les nombres d'occurences de deux objets.
    
    Passons ensuite aux différentes fonctions que nous avons créé.
    Premièrement, la méthode lire_fichier prend un paramètre un fichier, le lit, et renvoie ses caractères, sous forme d'une chaine de caractères.
    Ensuite, la méthode ecrire_fichier prend en paramètre le texte à écrire, et un fichier texte (par défaut texte.txt) dans lequel écrire ce texte.
    Puis, la méthode parcours prend en paramètre un arbre, une liste repréentant le parcours sur l'arbre, et un dictionnaire. Cette fonction fonctionne de manière récursive. Elle parcours l'arbre d'Huffman et définit les codes binaires des lettres du texte.
    
    Ensuite, la fonction fusionne prend en paramètres deux noeuds, gauche et droite, et renvoie un noeud ArbreHuffman en fusionnant le nombre d'occurences des noeux passés en paramètres.
    
    Puis, la fonction compte occurences prend en paramètre un texte, sous la forme d'une chaine de caractères, correspondant, dans le cas de notre projet, au texte extrait du fichier texte à compresser selon la méthode d'Huffman. Cette fonction compte le nombres d'occurences de chaque caractère, et les stocke dans un dictionnaire. On crée une nouvelle clé si et seulement si cette clé n'est pas déjà dans le dictionnaire. Sinon, on incrémente la valeur de la clé correspondant au caractère.
    
    De plus, la fonction construit_liste_arbres prend en paramètre le texte en chaine de caractères, et renvoie une liste d'objets ArbreHuffman créés à partir de chaque caractère, et de son occurence.
    
    Ensuite, la fonction codage_huffman prend en paramètre le texte sous forme de chaine de caractères, et renvoie un dictionnaire associant chaque caractère à son code binaire. Nous avons utilisé la méthode bisect (après avoir importé le module au préalable) afin d'ajouter un élément à la liste, le tout en la conservant triée.
    
    La fonction encode_message prend en paramètres le dictionnaire contenant les codes binaires de chaque caractère du texte, également passé en paramètre. Cette fonction encode le message, et le compresse selon la méthode de Huffman.

    Ensuite, la fonction sauvegarde prend en paramètre le résultat de la fonction encode_message, et le dictionnaire utile pour le décodage. Cette fonction sauvegarde le texte compressé dans le fichier huffman.txt, et le dictionnaire (au format JSON) dans le fichier décodage.txt.
    
    Puis, la fonction décrypte prend en paramètres les deux fichiers utiles pour le décodage (par défaut huffman.txt, et décodage.txt). Avec la lecture de ces fichiers, la fonction est capable de retrouver le texte de départ en parcourant le code binaire compressé.
    
    Enfin, toutes ces fonctions sont utilisées dans un ordre bien précis afin que tout fonctionne pour le mieux. Pour cela, l'utilisateur est interrogé plusieurs fois, et les réponses sont récupérées à l'aide des méthodes input. Il peut choisir l'encodage, ou le décodage. Il a également le choix d'écrire, ou non, dans le fichier texte de son choix. L'exécution se fait ensuite automatiquement.
    Si l'utilisateur choisit le décodage, il doit fournir les fichiers contenant le texte compressé, et le dictionnaire de décompression, s'il ne les a pas modifiés.
    Cette fonction d'execution se trouve dans le programme principal, à la suite de deux assertions vérifiant le bon fonctionnement du programme dans un cas précis. 